<a href="https://colab.research.google.com/github/vigneshiyer2803/Arm-Movement-detection-using-Mediapipe/blob/main/BERT_Untitled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df= pd.read_excel(r'/content/Moffit- Wordsmithing - Mapped Data.xlsx')
df=df.head(60000)

In [4]:
data = df[['Reason for Visit', 'Product Line', 'Sub Product Group']].dropna()
data

,Reason for Visit,Product Line,Sub Product Group
0,"CERVICAL MASS AND HX OF CERVICAL CA, SON CALLE...",GYN,GYN - Cervical
1,"THROMBOSIS EMBOLISM. PT CALLED, ALREADY EST AT...",Other,Other
2,GOITER ON THYROID GLAND US TAKEN PT CALLED SE...,Endocrine,Endo - Thyroid
3,R BREAST LUMP EST CARE SELF REF SPW PT FAX 8062,Breast,Breast
4,MM REF'D BY DR ALSINA FOR CAR T CONSULT,CAR T,CAR T
...,...,...,...
59995,PANCREATIC CA 2ND OP WEB CONSOLE SELF REF FAX,Non Neoplasm,Consult
59996,RECURRENT BREAST CA SELF REF TO EST CARE REQT ...,Breast,Breast
59997,ABN MAMMO AND US BIRAD 5 BREST BX NEEDED REF T...,Chemo/Radiation,Chemo/Radiation
59998,THYROID NODULES TIRADS 4 SPOKE WITH VALERIA C...,Endocrine,Endo - Thyroid


In [5]:
data['Label'] = data['Product Line'] + " ," + data['Sub Product Group']

In [6]:
data['Label']

0                     GYN ,GYN - Cervical
1                            Other ,Other
2               Endocrine ,Endo - Thyroid
3                          Breast ,Breast
4                            CAR T ,CAR T
                       ...               
59995               Non Neoplasm ,Consult
59996                      Breast ,Breast
59997    Chemo/Radiation ,Chemo/Radiation
59998           Endocrine ,Endo - Thyroid
59999                  Thoracic ,Thoracic
Name: Label, Length: 59648, dtype: object

In [7]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [7]:
# !pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [8]:
from datasets import Dataset, DatasetDict

# Create datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine them into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})




In [9]:
# Load DistilBERT tokenizer
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['Reason for Visit'], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/47718 [00:00<?, ? examples/s]

Map:   0%|          | 0/11930 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Reason for Visit', 'Product Line', 'Sub Product Group', 'Label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 47718
    })
    test: Dataset({
        features: ['Reason for Visit', 'Product Line', 'Sub Product Group', 'Label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 11930
    })
})

In [11]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
train_df['Label'] = label_encoder.fit_transform(train_df['Label'])
test_df['Label'] = label_encoder.transform(test_df['Label'])

# Update datasets with encoded labels
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# Tokenize again with labels included
def preprocess_function_with_labels(examples):
    result = tokenizer(examples['Reason for Visit'], padding='max_length', truncation=True)
    result['labels'] = examples['Label']
    return result

tokenized_dataset = dataset.map(preprocess_function_with_labels, batched=True)


Map:   0%|          | 0/47718 [00:00<?, ? examples/s]

Map:   0%|          | 0/11930 [00:00<?, ? examples/s]

In [12]:
train_df['Label']

24601    34
59607    46
58088    46
25492    34
33804     2
         ..
54665    12
38386    34
863      43
15888    43
56759     8
Name: Label, Length: 47718, dtype: int64

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Number of labels
num_labels = len(label_encoder.classes_)

# Load DistilBERT for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    save_strategy='epoch',
    save_steps=500,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # Start with 5 epochs
    weight_decay=0.01,
    logging_dir='./logs',
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",  # Assuming accuracy is the primary metric
    greater_is_better=True,
    eval_steps=500,  # Evaluate every 500 steps
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


In [18]:
# !pip install transformers[torch] accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)
